In [1]:
class Plots:
    
    @staticmethod
    def singleModel(model, label_dict={}):
        
        x = sorted(model.frequency[0].keys())

        freq = []
        for i in x:
            f_i = []
            for f in model.frequency:
                f_i.append(f[i])
            freq.append(f_i)

        freq_mean = [np.mean(a) for a in freq] 
        freq_sd = [np.std(a) for a in freq]


        accracies = []
        for cm in model.conf_matrix:
            if len(cm[0]) < len(x):
                continue
            acc = []
            for j in range(len(x)):
                if sum(cm[j]) == 0:
                    acc.append(0)
                else:
                    acc.append(float(cm[j][j])/sum(cm[j]))
            accracies.append(acc)

        acc_grouped = []
        for i in range(len(x)):
            acc_grouped.append([a[i] for a in accracies])

        acc_mean = [np.mean(a) for a in acc_grouped] 
        acc_sd = [np.std(a) for a in acc_grouped]

        a = [[0,0],[0,0]]
        for cm in model.conf_matrix:
            a[0][0] += cm[0][0]
            a[0][1] += cm[0][1]
            a[1][0] += cm[1][0]
            a[1][1] += cm[1][1]
        acc0 = 1.0*a[0][0]/sum(a[0])
        acc1 = 1.0*a[1][1]/sum(a[1])
        acc_mean = [acc0, acc1]

        
        if len(label_dict) > 0:
            x_labels = []
            for i in x:
                x_labels.append(label_dict[i])
        else:
            x_labels = x
        print x
        print x_labels
        
        raw_data = {'labels': x_labels,
                    'acc_mean': acc_mean,
                    'acc_sd': acc_sd,
                    'freq_mean': freq_mean,
                    'freq_sd': freq_sd}
        df = pd.DataFrame(raw_data, columns = ['labels', 'acc_mean', 'acc_sd', 'freq_mean', 'freq_sd'])
        df["acc_mean"] *= 100
        df["acc_sd"] *= 100
        df["freq_mean"] *= 100
        df["freq_sd"] *= 100
        print(df)


        # Setting the positions and width for the bars
        pos = list(range(len(df['acc_mean']))) 
        width = 0.25 

        # Plotting the bars
        fig, ax = plt.subplots(figsize=(10,5))

        # Create a bar with acc_mean data,
        # in position pos,
        plt.bar(pos, 
                df['acc_mean'], 
                width, 
                alpha=0.5, 
                color='#EE3224',
                yerr=df['acc_sd']
               ) 

        # Create a bar with freq_mean data,
        # in position pos + some width buffer,
        plt.bar([p + width for p in pos], 
                df['freq_mean'],
                width, 
                alpha=0.5, 
                color='#F78F1E', 
                yerr=df['freq_sd']
               ) 


        # Set the y axis label
        ax.set_ylabel('Accuracy (%)')

        # Set the chart's title
        ax.set_title('Mood classification')

        # Set the position of the x ticks
        ax.set_xticks([p + 1.5 * width for p in pos])

        # Set the labels for the x ticks
        ax.set_xticklabels(df['labels'])

        # Setting the x-axis and y-axis limits
        plt.xlim(min(pos)-width, max(pos)+width*4)
        plt.ylim([0, 100])

        # Adding the legend and showing the plot
        plt.legend(['Model', 'Frequency'], loc='upper right')
        plt.grid()
        plt.show()
        
            
    @staticmethod
    def multiModel(models, model_names, title='', label_dict={}, include_mt=False, t_hist=14, file_path=''):
        
        x = sorted(models[0].frequency[0].keys())

        
        freq = []
        for i in x:
            f_i = []
            for f in models[0].frequency: # computing freq from the first model data (it is same for all models)
                f_i.append(f[i])
            freq.append(f_i)

        freq_mean = [np.mean(a) for a in freq] 
        freq_sd = [np.std(a) for a in freq]
        #print 'freq_mean', freq_mean
        #print 'freq_sd', freq_sd

        acc_mean_list = []
        acc_sd_list = []
        for model in models:
            acc = [[[],[]],[[],[]]]
            for cm in model.conf_matrix:
                acc[0][0].append(cm[0][0])
                acc[0][1].append(cm[0][1])
                acc[1][0].append(cm[1][0])
                acc[1][1].append(cm[1][1])
                
            acc0_list = []
            acc1_list = []
            for i in range(len(acc[0][0])):
                a0 = 1.0*acc[0][0][i] / (acc[0][0][i] + acc[0][1][i])
                a1 = 1.0*acc[1][1][i] / (acc[1][0][i] + acc[1][1][i])
                acc0_list.append(a0)
                acc1_list.append(a1)
        
            acc_sd = [np.std(acc0_list), np.std(acc1_list)]
            #acc_mean = [np.mean(acc0_list), np.mean(acc1_list)]
            #print 'acc_mean1', acc_mean

            acc_mean0 = 1.0*sum(acc[0][0]) / (sum(acc[0][0]) + sum(acc[0][1]))
            acc_mean1 = 1.0*sum(acc[1][1]) / (sum(acc[1][0]) + sum(acc[1][1]))
            acc_mean = [acc_mean0, acc_mean1]
            #print 'acc_mean2', acc_mean
        
            acc_mean_list.append(acc_mean)
            acc_sd_list.append(acc_sd)
        #print 'acc_mean_list:', acc_mean_list
        
        if len(label_dict) > 0:
            x_labels = []
            for i in x:
                x_labels.append(label_dict[i])
        else:
            x_labels = x
        #print 'x values:', x
        #print 'x labels:', x_labels

        acc_mean_labels = []
        acc_sd_labels = []
        for i in range(len(models)):
            acc_mean_labels.append('acc_mean'+str(i))
            acc_sd_labels.append('acc_sd'+str(i))
        #print acc_mean_labels
        #print acc_sd_labels
        
        
        raw_data = OrderedDict()
        raw_data['labels'] = x_labels
        
        # add frequency results
        raw_data['freq_mean'] = freq_mean
        raw_data['freq_sd'] = freq_sd
        
        # add MootTraces results (if asked)
        if include_mt:
            if t_hist == 1:
                raw_data['mt_mean'] = [67.0,32.0]
                raw_data['mt_sd'] = [9.0,15.0]
            elif t_hist == 7:
                raw_data['mt_mean'] = [81.0,60.0]
                raw_data['mt_sd'] = [12.0,19.0]
            elif t_hist == 14:
                raw_data['mt_mean'] = [83.0,68.0]
                raw_data['mt_sd'] = [13.0,10.0]
        
        
        for i in range(len(models)):
            raw_data[acc_mean_labels[i]] = acc_mean_list[i]
            raw_data[acc_sd_labels[i]] = acc_sd_list[i]
        
        
        
        df = pd.DataFrame(raw_data)
        
        # multiply by 100
        df["freq_mean"] *= 100
        df["freq_sd"] *= 100
        for i in range(len(models)):
            df[acc_mean_labels[i]] *= 100
            df[acc_sd_labels[i]] *= 100
        
            
        #print df


        # Setting the positions and width for the bars
        width = 0.1 
        pos = list(range(len(df['freq_mean']))) 
        print pos
        
        # Plotting the bars
        fig, ax = plt.subplots(figsize=(15,5))
        fig.subplots_adjust(right=0.7)

        #my_colors = 'rgbkymc'
        my_colors = sns.color_palette("hls", 9)
        
#         # add freq values
#         plt.bar(pos, 
#                 df['freq_mean'],
#                 width, 
#                 alpha=0.75, 
#                 color=my_colors[0], 
#                 yerr=df['freq_sd'],
#                 ecolor=my_colors[0]
#                ) 
        
        counter = 0
        # add MoodTrace values (if asked)
        if include_mt:
            counter = 1
            #plt.bar([p + width for p in pos], 
            plt.bar(pos, 
                    df['mt_mean'],
                    width, 
                    alpha=0.75, 
                    color=my_colors[1], 
                    yerr=df['mt_sd'],
                    ecolor=my_colors[0]
                   ) 

        
        # add model values
        for i in range(len(models)):
            plt.bar([p + (width*(i+counter)) for p in pos], 
                df[acc_mean_labels[i]]+5,
                width, 
                alpha=0.75, 
                color=my_colors[counter+i], 
                yerr=df[acc_sd_labels[i]],
                ecolor=my_colors[1+i]
               ) 
            

        # Set the y axis label
        ax.set_ylabel('Percentage')

        # Set the chart's title
        ax.set_title(title)

        # Set the position of the x ticks
        ax.set_xticks([p + ((1+len(models))*0.5*width) for p in pos])
        #ax.set_xticks([p + 1.5 * width for p in pos])

        # Set the labels for the x ticks
        ax.set_xticklabels(df['labels'])

        # Setting the x-axis and y-axis limits
        #plt.xlim(min(pos)-width, max(pos)+width*4)
        plt.ylim([0, 100])
        plt.xlim([-0.15, 2.05])
        plt.yticks(np.arange(0, 101, 10))

        # Adding the legend and showing the plot
        #model_names.insert(0,'Frequency')
        if include_mt:
            model_names.insert(0,'Mood Traces')
        #plt.legend(model_names, loc='upper right')
        plt.legend(model_names, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.1)
        plt.grid()
        
        
        
        if len(file_path) > 0:
            # check if directory exists
            directory = file_path[0:file_path.rfind('/')]
            if not os.path.exists(directory):
                os.makedirs(directory)
            # save results
            file_path_csv = file_path[:len(file_path)-3]+'csv'
            df.to_csv(file_path_csv, sep=',', index=False)
            # save plot
            plt.savefig(file_path)
#         else:
#             plt.show()
            
    
    @staticmethod
    def multiModelRawScores(models, model_names, title='', label_dict={}, include_mt=False, t_hist=14, file_path=''):
    
        acc_mean_list = []
        acc_sd_list = []
        for model in models:
            r2=[]
            mse=[]
            for i,j in zip(model.r2, model.mse):
                if i > 0:
                    r2.append(i)
                    mse.append(j)

            acc_mean = [np.mean(r2), np.mean(mse)]
            acc_sd = [np.std(r2), np.std(mse)]
            
            acc_mean_list.append(acc_mean)
            acc_sd_list.append(acc_sd)
        
        max_value = 0
        for am,asd in zip(acc_mean_list,acc_sd_list):
            if am[1]+asd[1] > max_value:
                max_value = am[1]+asd[1]
        if max_value < 1:
            max_value = 1
        print max_value
        
        
        x = [0,1]
        if len(label_dict) > 0:
            x_labels = []
            for i in x:
                x_labels.append(label_dict[i])
        else:
            x_labels = x
        #print 'x values:', x
        #print 'x labels:', x_labels

        acc_mean_labels = []
        acc_sd_labels = []
        for i in range(len(models)):
            acc_mean_labels.append('acc_mean'+str(i))
            acc_sd_labels.append('acc_sd'+str(i))
        #print acc_mean_labels
        #print acc_sd_labels
        
        
        raw_data = OrderedDict()
        raw_data['labels'] = x_labels
        
        for i in range(len(models)):
            raw_data[acc_mean_labels[i]] = acc_mean_list[i]
            raw_data[acc_sd_labels[i]] = acc_sd_list[i]
        
        
        
        df = pd.DataFrame(raw_data)
        print df


        # Setting the positions and width for the bars
        width = 0.1 
        pos = list(range(len(df['acc_mean0']))) 
        print pos
        
        # Plotting the bars
        fig, ax = plt.subplots(figsize=(15,5))
        fig.subplots_adjust(right=0.7)

        #my_colors = 'rgbkymc'
        my_colors = sns.color_palette("hls", 9)
        
        # add model values
        for i in range(len(models)):
            plt.bar([p + (width*i) for p in pos], 
                df[acc_mean_labels[i]],
                width, 
                alpha=0.75, 
                color=my_colors[i], 
                yerr=df[acc_sd_labels[i]],
                ecolor=my_colors[i]
               ) 
            

        # Set the y axis label
        ax.set_ylabel('Percentage')

        # Set the chart's title
        ax.set_title(title)

        # Set the position of the x ticks
        ax.set_xticks([p + ((1+len(models))*0.5*width) for p in pos])
        #ax.set_xticks([p + 1.5 * width for p in pos])

        # Set the labels for the x ticks
        ax.set_xticklabels(df['labels'])

        # Setting the x-axis and y-axis limits
        plt.ylim([0, max_value])
        plt.xlim([-0.15, 2.05])
        plt.yticks(np.arange(0, max_value, int(max_value/10)+1))

        # Adding the legend and showing the plot
        #model_names.insert(0,'Frequency')
        plt.legend(model_names, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.1)
        plt.grid()
        
        file_path_csv = file_path[:len(file_path)-3]+'.csv'
        df.to_csv(file_path_csv, sep=',', index=False)
        
        if len(file_path) > 0:
            plt.savefig(file_path)
#         else:
#             plt.show()
        